***
***INSTALLINGS AND IMPORTINGS***
***

* Installings

In [ ]:
# INSTALLINGS

!pip install imagecodecs
!pip install histomicstk --find-links https://girder.github.io/large_image_wheels 
!pip install opencv-python-headless==4.1.2.30
!pip install pyyaml==5.4.1

     |████████████████████████████████| 31.0 MB 1.3 MB/s 
Looking in links: https://girder.github.io/large_image_wheels
     |████████████████████████████████| 534 kB 3.0 MB/s 
     |████████████████████████████████| 19.4 MB 13.6 MB/s 
     |████████████████████████████████| 57 kB 5.2 MB/s 
     |████████████████████████████████| 4.7 MB 26.4 MB/s 
     |████████████████████████████████| 47.7 MB 1.4 MB/s 
     |████████████████████████████████| 170 kB 44.8 MB/s 
     |████████████████████████████████| 134 kB 53.0 MB/s 
     |████████████████████████████████| 44 kB 3.3 MB/s 
     |████████████████████████████████| 54 kB 3.2 MB/s 
     |████████████████████████████████| 41.4 MB 1.1 MB/s 
     |████████████████████████████████| 829 kB 46.9 MB/s 
     |████████████████████████████████| 65.3 MB 52 kB/s 
     |████████████████████████████████| 132 kB 62.3 MB/s 
     |████████████████████████████████| 8.6 MB 36.8 MB/s 
     |████████████████████████████████| 79 kB 8.3 MB/s 
     |█████████████

* Importings

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import numpy as np
import histomicstk as htk  # notice this line will give an error the first time 
                           # the cell is runned, please just re-run the cell and
                           # the problem will automatically solved (there is 
                           # probably some kind of bug)
import imagecodecs
import scipy

from keras.models import load_model
from skimage import morphology
from scipy import ndimage 
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from tqdm import tqdm
from skimage.io import imread, imsave
from skimage.transform import resize
from skimage import img_as_ubyte
from skimage.morphology import binary_dilation, remove_small_objects, remove_small_holes,binary_closing,binary_opening

Mounted at /content/drive


* Loading data and trained models in Colab 

In [ ]:
# DATA LOADING

!pip install unrar

# manual masks 
!unrar x "drive/MyDrive/EIM challenge_gruppo_FA_DO_PA_PA/00_DATASET/train.rar"        # unraring training set
!unrar x "drive/MyDrive/EIM challenge_gruppo_FA_DO_PA_PA/00_DATASET/validation.rar"   # unraring validation set
!unrar x "drive/MyDrive/EIM challenge_gruppo_FA_DO_PA_PA/00_DATASET/test.rar"         # unraring test set

# indagated net
indagated_net1 = 'x1'
indagated_net2 = 'x2'
indagated_net3 = 'x3'
model1 = load_model('drive/MyDrive/EIM challenge_gruppo_FA_DO_PA_PA/02_TRAINED MODELS/' + indagated_net1) 
model2 = load_model('drive/MyDrive/EIM challenge_gruppo_FA_DO_PA_PA/02_TRAINED MODELS/' + indagated_net2)   
model3 = load_model('drive/MyDrive/EIM challenge_gruppo_FA_DO_PA_PA/02_TRAINED MODELS/' + indagated_net3)
models = [model1,model2,model3]

# resizing setting
rsz = 512

# reference image for Reihnard normalization
img_ref = imread('train/images/104.tif')
img_ref = img_as_ubyte(resize(img_ref,[rsz,rsz]))  # mean and std for color normalization are calculated after resizing to 512x512
mean_ref, std_ref = htk.preprocessing.color_conversion.lab_mean_std(img_ref)


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from drive/MyDrive/EIM challenge_gruppo_FA_DO_PA_PA/00_DATASET/train.rar

Creating    train                                                     OK
Creating    train/images                                              OK
Extracting  train/images/102.tif                                           0%  OK 
Extracting  train/images/104.tif                                           0%  1%  OK 
Extracting  train/images/106.tif                                           1%  OK 
Extracting  train/images/108.tif                                           1%  2%  OK 
Extracting  train/images/109.tif                                           2%  OK 
Extracting  train/images/111.tif                                           2%  3%  OK 
Extracting  train/images/112.tif                                           3%  OK 
Extracting  train/images/114.tif

***
# ***OBTAINING MASKS***
***

In [ ]:
# CREATING DIRECTORIES IN WHICH TO STORE THE PREDICTED MASK

save_tr = "train_auto"
if not os.path.exists(save_tr):
  os.mkdir(save_tr)

save_vl = "validation_auto"
if not os.path.exists(save_vl):
  os.mkdir(save_vl)

save_ts = "test_auto"
if not os.path.exists(save_ts):
  os.mkdir(save_ts)

***
TESTING
***


* Training set

In [ ]:
# PREDICTIONS ON TRAINING SET

# path
tr_IMGS_path = os.path.join('train','images')  # path to original image
tr_MANU_path = os.path.join('train','manual')  # path to manual annotations

# list of all images on which to apply the U-NET
tr_images = os.listdir(tr_IMGS_path)

# body
perf_tr = np.zeros((0,3))  # initialization of performances array
missed_cells_tr = []  # not found cells
erroneous_cells_tr = []  # "invented" cells
dinofcep_tr = []  # difference in the number of cells for each patch
for id_ in tqdm(tr_images, total=len(tr_images)):
    
    # load and resizing of stained image
    img = imread(tr_IMGS_path+'/'+id_)
    ori_shape = img.shape
    img = img_as_ubyte(resize(img,(rsz,rsz)))

    # color normalization
    img = htk.preprocessing.color_normalization.reinhard(img, mean_ref, std_ref)

    # apply ensembled models for prediction (ensembling is done at probability map level)
    hm = np.zeros((img.shape[0],img.shape[1],2),dtype=np.float32)
    for i in range(len(models)):
        hm = hm + models[i].predict(img[np.newaxis,:])[0,:,:,1:]  # notice reshaping of img to feed for net format is done here in fly  
    
    # post-processing
    ## post-processimg: thresholding
    mask0 = np.zeros((hm.shape[0],hm.shape[1]),dtype=np.uint8)
    mask0[hm[:,:,0]>1.35] = 128
    mask0[hm[:,:,1]>1.35] = 255  # predictions on nuclei are more accurate in terms of edges, so it is ok to eventually overwrite cytoplasm segmentations w/ nucleus segmentation
    ## post-processing: morphological refinement operations
    mask0 = morphology.area_closing(mask0,area_threshold=200)  # small holes removal
    mask0 = morphology.area_opening(mask0,area_threshold=100)  # small objects removal
    ## post-processing: resizing to original shape
    mask0 = img_as_ubyte(resize(mask0,(ori_shape[0],ori_shape[1])))
    mask0[mask0<80] = 0
    mask0[(mask0>=80)*(mask0<=175)] = 128
    mask0[mask0>175] = 255
    ## post-processing: removal of too small nuclei 
    nuclei,_ = ndimage.label(mask0==255)
    for n in range(1,nuclei.max()+1):
      if (nuclei==n).sum() < 0.003*mask0.shape[0]*mask0.shape[1]: mask0[nuclei==n] = 0
    ## post-processing: eventual watershed (hypothesis of each cytoplasm being 
    ## connected to corresponding nuclues in at least 1 pixel is done) and 
    ## storing each segmented cell on a different layer
    mask = np.zeros((mask0.shape[0],mask0.shape[1],0),dtype=np.uint8)  # initialization of the final N-layer mask
    labeled, n_trees = ndimage.label((mask0==128)+(mask0==255))  # finding connected components of thresholded mask
    for i in range(1,n_trees+1):
      _,n_nuc = ndimage.label((labeled==i)*(mask0==255))  # finding number of nuclei in the currently passed connected component (CC)
      if n_nuc > 1:   # case in which there is than 1 nucleus in the current CC, then the CC may be a cluster of cells --> watersheding is applied
        dist_transform = ndimage.distance_transform_edt(labeled==i)  # distance transform of the CC
        dst_for_mark = ndimage.distance_transform_edt((labeled==i)*(mask0==255)) # distance trasform on nuclei of the CC
        lab_for_mark,_ = ndimage.label((labeled==i)*(mask0==255))  # labeling nuclei of the CC
        mark_coords = peak_local_max(dst_for_mark, labels=lab_for_mark, num_peaks_per_label=1)  # finding markers' coordinate for subsequent watershed
        markers = np.zeros(dist_transform.shape, dtype=bool); markers[tuple(mark_coords.T)] = True; markers, _ = ndimage.label(markers)  # markers
        separated = watershed(-dist_transform, markers, mask=labeled==i)  # actual watersheding
        ### nuclei restoring (watersheding could have -erroneously- cut in more 
        ### parts some nucleus)
        restore = False  # nuclei restoring flag
        for j in range(1,separated.max()+1):  # cycling on the identified cells of the cluster
          curr = np.zeros_like(mask0); curr[separated==j] = mask0[separated==j] # segmentation of current cell
          mask = np.append(mask,curr[:,:,np.newaxis],axis=2)
          #### checking if nuclei restoring is needed 
          _,n_nuc2 = ndimage.label(curr==255)
          if n_nuc2 > 1: restore = True 
        if restore:   # if restoring is needed
          rank = np.zeros((n_nuc,n_nuc))  # array that will contain ranking of appartenence for each nuclei (on columns) to each watersheded object
          #### nuclei-matching ranking
          for k in range(1,n_nuc+1):  # loop over nuclei
             for l in range(1,n_nuc+1):  # loop over watersheded objects
               rank[l-1,k-1] = ((lab_for_mark==k)*(mask[:,:,-l]==255)).sum()/(lab_for_mark==k).sum()  
          rank = np.argmax(rank,0)+1  # array containing for each nucleus the watersheded object containing it at greatest %
          #### actual restoring
          for k in range(n_nuc):
            mask[lab_for_mark==k+1,-rank[k]] = 255
            mask[lab_for_mark==k+1,-1:-rank[k]:-1] = 0; mask[lab_for_mark==k+1,-rank[k]-1:-n_nuc-1:-1] = 0;
      else:  # case in which watersheding is not needed
        curr = np.zeros_like(mask0); curr[labeled==i] = mask0[labeled==i] # segmentation of current cell
        mask = np.append(mask,curr[:,:,np.newaxis],axis=2)
    
    # removing layers in which no nucleus has been found (minimum requirement to
    # have a segmentation is presence of nucleus), with this step also initial 
    # empty layer initialization is removed 
    w2m = np.sum(mask==255,axis=(0,1)).astype(bool); mask = mask[:,:,w2m]  
    
    # if no cells have been segmneted, an empty single layer mask is saved [2-D 
    # empty mask], else if only one cell has been found, then the mask is 
    # "compressed" to 2-D  
    if mask.size == 0:  # no cell case
      mask = np.zeros((mask0.shape[0],mask0.shape[1]),dtype=np.uint8)
    elif mask.shape[2] == 1:  # 1 cell case
      mask = np.squeeze(mask)

    # saving mask: please, to further  read automatic annotations use 'imread' 
    # method of skimage library and not the one of imagecodecs library 
    imsave(os.path.join(save_tr,id_),mask,check_contrast=False)  

    # calculating performances at single-cell mask level: please notice in this
    # implementation intersection over union (IoU) at whole cell level has been 
    # used to match manual single layer mask with its correspective automatic 
    # mask in order to calculate performances (IoU nucleus, IoU cytoplasm, IoU 
    # whole cell)
    ## load N-D image containing manual segmentations (each layer a different MM cell, 128 = cytoplasm | 255 = nucleus) 
    manu = imagecodecs.imread(tr_MANU_path+'/'+id_) 
    ## defining number of layers of manual mask and automatic mask
    try:
      n_layer_manu = manu.shape[2]
    except:  # case in which no cells or only 1 cell has been segmented
      if manu.sum()==0:  # no found cell case
        n_layer_manu = 0
      else:  # one cell case
        n_layer_manu = 1
        manu = np.expand_dims(manu,2)  # dimensions expansion is needed to mantain a unificate code also in case of only 1 cell segmentation
    try:
      n_layer_auto = mask.shape[2]
    except:  # case in which no cells or only 1 cell has been segmented
      if mask.sum()==0:  # no found cell case
        n_layer_auto = 0
      else:  # one cell case
        n_layer_auto = 1
        mask = np.expand_dims(mask,2)  # dimensions expansion is needed to mantain a unificate code also in case of only 1 cell segmentation
    ## current patch performance calculation
    perf = np.zeros((max(n_layer_manu,n_layer_auto),3))  # array that will contain performances for the current patch mask-matching predictions 
    if n_layer_manu and n_layer_auto:  # if at least one cell has been segmented in both manual mask and automatic mask
      IoU_WC = np.zeros((n_layer_auto,n_layer_manu))  # array that will contain IoU of whole cell all possible combinations of 'single layer manual mask' | 'single layer predicted mask'
      ### whole cell level IoU of all possible combinations
      for o1 in range(n_layer_manu):
        for o2 in range(n_layer_auto):
          IoU_WC[o2,o1] = ((manu[:,:,o1]).astype(bool)*(mask[:,:,o2]).astype(bool)).sum()/((manu[:,:,o1]).astype(bool)+(mask[:,:,o2]).astype(bool)).sum()  #IoU of current combination 
      ### case in which automatic prediction found at least equal number of 
      ### cells of those identified in the manual annotations
      if n_layer_auto >= n_layer_manu: 
        order = np.argmax(IoU_WC,axis=0)  # finding most matching automatically segmented cell mask for each manually segmented cell
        #### solving manual mask - automatic mask matching redundancies: it may happen that one single-cell segmetation of one of the two types is matched with more than one single-cell segmentation of the other type, but obviously matchings have to be non redundant
        _,inv_ind,counts = np.unique(order,return_inverse=True, return_counts=True)
        if (counts>1).any():
          for a in range(max(inv_ind)+1):
            if (inv_ind==a).sum() > 1:
              w2d = (inv_ind==a)*~(IoU_WC.max(axis=0)==max(IoU_WC.max(axis=0)[inv_ind==a]))
              order = np.delete(order,w2d)
              inv_ind = np.delete(inv_ind,w2d)
              IoU_WC = np.delete(IoU_WC,w2d,1)
              manu = manu[:,:,~w2d]
        IoU_WC = np.max(IoU_WC,axis=0);  # IoU whole cell for all matched cell pairs
        IoU_N = np.sum((manu==255)*(mask[:,:,order]==255),axis=(0,1))/np.sum((manu==255)+(mask[:,:,order]==255),axis=(0,1))  # IoU nucleus for all matched cell pairs
        IoU_C = np.sum((manu==128)*(mask[:,:,order]==128),axis=(0,1))/np.sum((manu==128)+(mask[:,:,order]==128),axis=(0,1))  # IoU cytoplasm for all matched cell pairs
      ### case in which automatic prediction found a smaller number of cells of 
      ### those identified in the manual annotations
      else:
        order = np.argmax(IoU_WC,axis=1)
        #### solving manual mask - automatic mask matching redundancies: it may happen that one single-cell segmetation of one of the two types is matched with more than one single-cell segmentation of the other type, but obviously matchings have to be non redundant
        _,inv_ind,counts = np.unique(order,return_inverse=True, return_counts=True)
        if (counts>1).any():
          w2m_WC = np.ones(len(order),dtype=bool)
          for a in range(max(inv_ind)+1):
            if (inv_ind==a).sum() > 1:
              w2d = (inv_ind==a)*~(IoU_WC.max(axis=1)==max(IoU_WC.max(axis=1)[inv_ind==a]))
              order = np.delete(order,w2d)
              inv_ind = np.delete(inv_ind,w2d)
              IoU_WC = np.delete(IoU_WC,w2d,0)
              mask = mask[:,:,~w2d]
        IoU_WC = np.max(IoU_WC,axis=1)  
        IoU_N = np.sum((manu[:,:,order]==255)*(mask==255),axis=(0,1))/np.sum((manu[:,:,order]==255)+(mask==255),axis=(0,1))  
        IoU_C = np.sum((manu[:,:,order]==128)*(mask==128),axis=(0,1))/np.sum((manu[:,:,order]==128)+(mask==128),axis=(0,1))  
      perf = np.stack((IoU_N.T,IoU_C.T,IoU_WC.T),axis=1)  # performances on matched cells 
    erroneous_cells_tr.append(n_layer_auto-(perf[:,0]>0).sum()) 
    missed_cells_tr.append(n_layer_manu-(perf[:,0]>0).sum())
    dinofcep_tr.append(abs(n_layer_manu-n_layer_auto))
    perf_tr = np.concatenate((perf_tr,perf,np.zeros((erroneous_cells_tr[-1]+missed_cells_tr[-1],3))),axis=0)  # adding current patch performances to global ones    

# performances printing
cell_number_diff_tr = np.array((erroneous_cells_tr,missed_cells_tr,dinofcep_tr),dtype=np.int8).T
print(f"\nPERFORMNACES ON TRAINING SET as IoU nuclei | IoU cytoplasm | IoU whole cell:\n\n{perf_tr}\n\nmean and std: {np.mean(perf_tr,axis=0)} and {np.std(perf_tr,axis=0)}")
print(f"\n\n\n\nINVENTED CELLS | MISSED CELLS | DIFFERENCE IN # OF SEGMNETED CELLS:\n\n{cell_number_diff_tr}\n\n mean: {np.mean(cell_number_diff_tr,axis=0)}")
print(f"\n\n\n\nPERFORMNACES only on currectly detected MM cells as IoU nuclei | IoU cytoplasm | IoU whole cell:\n\nmean and std: {np.mean(perf_tr[perf_tr[:,0]>0,:],axis=0)} and {np.std(perf_tr[perf_tr[:,0]>0,:],axis=0)}")

100%|██████████| 300/300 [50:02<00:00, 10.01s/it]


PERFORMNACES ON TRAINING SET as IoU nuclei | IoU cytoplasm | IoU whole cell:

[[0.89767961 0.85539793 0.92375157]
 [0.97589363 0.91610695 0.95928874]
 [0.94702049 0.54062682 0.68753885]
 ...
 [0.32782867 0.36721611 0.35801673]
 [0.         0.         0.        ]
 [0.         0.         0.        ]]

mean and std: [0.82925534 0.7223226  0.80922619] and [0.29773385 0.29130554 0.29274753]




INVENTED CELLS | MISSED CELLS | DIFFERENCE IN # OF SEGMNETED CELLS:

[[2 1 1]
 [0 0 0]
 [2 1 1]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [1 0 1]
 [1 0 1]
 [0 0 0]
 [0 0 0]
 [1 1 0]
 [1 0 1]
 [1 0 1]
 [1 0 1]
 [0 1 1]
 [1 0 1]
 [0 0 0]
 [0 0 0]
 [1 0 1]
 [2 2 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [1 1 0]
 [0 2 2]
 [0 0 0]
 [0 0 0]
 [2 0 2]
 [0 1 1]
 [3 0 3]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [2 1 1]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 1 1]
 [0 0 0]
 [1 0 1]
 [0 0 0]
 [1 0 1]
 [2 1 1]
 [1 0 1]
 [0 0 0]
 [0 0 0]
 [0 1 1]
 [2 1 1]
 [0 1 1]
 [0 2 2]
 [1 0 1]
 [1 0 1]
 [0 0 0]
 [0 0 0]
 [1 1 0]
 [1 1 0]
 [0 1 

* Validation set

In [ ]:
# PREDICTIONS ON VALIDATION SET

# Path
vl_IMGS_path = os.path.join('validation','images')  # path to originale images
vl_MANU_path = os.path.join('validation','manual')  # path to manual annotations                             

# list of all images on which to apply the U-NET
vl_images = os.listdir(vl_IMGS_path)

# body
perf_vl = np.zeros((0,3))  # initialization of performances array
missed_cells_vl = []  # not found cells
erroneous_cells_vl = []  # "invented" cells
dinofcep_vl = []  # difference in the number of cells for each patch
for id_ in tqdm(vl_images, total=len(vl_images)):
    
    # load and resizing of stained image
    img = imread(vl_IMGS_path+'/'+id_)
    ori_shape = img.shape
    img = img_as_ubyte(resize(img,(rsz,rsz)))

    # color normalization
    img = htk.preprocessing.color_normalization.reinhard(img, mean_ref, std_ref)

    # apply ensembled models for prediction (ensembling is done at probability map level)
    hm = np.zeros((img.shape[0],img.shape[1],2),dtype=np.float32)
    for i in range(len(models)):
        hm = hm + models[i].predict(img[np.newaxis,:])[0,:,:,1:]  # notice reshaping of img to feed for net format is done here in fly  
    
    # post-processing
    ## post-processimg: thresholding
    mask0 = np.zeros((hm.shape[0],hm.shape[1]),dtype=np.uint8)
    mask0[hm[:,:,0]>1.35] = 128
    mask0[hm[:,:,1]>1.35] = 255  # predictions on nuclei are more accurate in terms of edges, so it is ok to eventually overwrite cytoplasm segmentations w/ nucleus segmentation
    ## post-processing: morphological refinement operations
    mask0 = morphology.area_closing(mask0,area_threshold=200)  # small holes removal
    mask0 = morphology.area_opening(mask0,area_threshold=100)  # small objects removal
    ## post-processing: resizing to original shape
    mask0 = img_as_ubyte(resize(mask0,(ori_shape[0],ori_shape[1])))
    mask0[mask0<80] = 0
    mask0[(mask0>=80)*(mask0<=175)] = 128
    mask0[mask0>175] = 255
    ## post-processing: removal of too small nuclei 
    nuclei,_ = ndimage.label(mask0==255)
    for n in range(1,nuclei.max()+1):
      if (nuclei==n).sum() < 0.003*mask0.shape[0]*mask0.shape[1]: mask0[nuclei==n] = 0
    ## post-processing: eventual watershed (hypothesis of each cytoplasm being 
    ## connected to corresponding nuclues in at least 1 pixel is done) and 
    ## storing each segmented cell on a different layer
    mask = np.zeros((mask0.shape[0],mask0.shape[1],0),dtype=np.uint8)  # initialization of the final N-layer mask
    labeled, n_trees = ndimage.label((mask0==128)+(mask0==255))  # finding connected components of thresholded mask
    for i in range(1,n_trees+1):
      _,n_nuc = ndimage.label((labeled==i)*(mask0==255))  # finding number of nuclei in the currently passed connected component (CC)
      if n_nuc > 1:   # case in which there is than 1 nucleus in the current CC, then the CC may be a cluster of cells --> watersheding is applied
        dist_transform = ndimage.distance_transform_edt(labeled==i)  # distance transform of the CC
        dst_for_mark = ndimage.distance_transform_edt((labeled==i)*(mask0==255)) # distance trasform on nuclei of the CC
        lab_for_mark,_ = ndimage.label((labeled==i)*(mask0==255))  # labeling nuclei of the CC
        mark_coords = peak_local_max(dst_for_mark, labels=lab_for_mark, num_peaks_per_label=1)  # finding markers' coordinate for subsequent watershed
        markers = np.zeros(dist_transform.shape, dtype=bool); markers[tuple(mark_coords.T)] = True; markers, _ = ndimage.label(markers)  # markers
        separated = watershed(-dist_transform, markers, mask=labeled==i)  # actual watersheding
        ### nuclei restoring (watersheding could have -erroneously- cut in more 
        ### parts some nucleus)
        restore = False  # nuclei restoring flag
        for j in range(1,separated.max()+1):  # cycling on the identified cells of the cluster
          curr = np.zeros_like(mask0); curr[separated==j] = mask0[separated==j] # segmentation of current cell
          mask = np.append(mask,curr[:,:,np.newaxis],axis=2)
          #### checking if nuclei restoring is needed 
          _,n_nuc2 = ndimage.label(curr==255)
          if n_nuc2 > 1: restore = True   
        if restore:   # if restoring is needed
          rank = np.zeros((n_nuc,n_nuc))  # array that will contain ranking of appartenence for each nuclei (on columns) to each watersheded object
          #### nuclei-matching ranking
          for k in range(1,n_nuc+1):  # loop over nuclei
             for l in range(1,n_nuc+1):  # loop over watersheded objects
               rank[l-1,k-1] = ((lab_for_mark==k)*(mask[:,:,-l]==255)).sum()/(lab_for_mark==k).sum()  
          rank = np.argmax(rank,0)+1  # array containing for each nucleus the watersheded object containing it at greatest %
          #### actual restoring
          for k in range(n_nuc):
            mask[lab_for_mark==k+1,-rank[k]] = 255
            mask[lab_for_mark==k+1,-1:-rank[k]:-1] = 0; mask[lab_for_mark==k+1,-rank[k]-1:-n_nuc-1:-1] = 0;
      else:  # case in which watersheding is not needed
        curr = np.zeros_like(mask0); curr[labeled==i] = mask0[labeled==i] # segmentation of current cell
        mask = np.append(mask,curr[:,:,np.newaxis],axis=2)
    
    # removing layers in which no nucleus has been found (minimum requirement to
    # have a segmentation is presence of nucleus)
    w2m = np.sum(mask==255,axis=(0,1)).astype(bool); mask = mask[:,:,w2m]  
    
    # if no cells have been segmneted, an empty single layer mask is saved [2-D 
    # empty mask], else if only one cell has been found, then the mask is 
    # "compressed" to 2-D  
    if mask.size == 0:  # no cell case
      mask = np.zeros((mask0.shape[0],mask0.shape[1]),dtype=np.uint8)
    elif mask.shape[2] == 1:  # 1 cell case
      mask = np.squeeze(mask)

    # saving mask: please, to read automatic annotations use 'imread' method of 
    # skimage library and not the one of imagecodecs library 
    imsave(os.path.join(save_vl,id_),mask,check_contrast=False)  


    # calculating performances at single-cell mask level: please notice in this
    # implementation intersection over union (IoU) at whole cell level has been 
    # used to match manual single layer mask with its correspective automatic 
    # mask in order to calculate performances (IoU nucleus, IoU cytoplasm, IoU 
    # whole cell)
    ## load N-D image containing manual segmentations (each layer a different MM cell, 128 = cytoplasm | 255 = nucleus) 
    manu = imagecodecs.imread(vl_MANU_path+'/'+id_)
    ## defining number of layers of manual mask and automatic mask
    try:
      n_layer_manu = manu.shape[2]
    except:  # case in which no cells or only 1 cell has been segmented
      if manu.sum()==0:  # no found cell case
        n_layer_manu = 0
      else:  # one cell case
        n_layer_manu = 1
        manu = np.expand_dims(manu,2)  # dimensions expansion is needed to mantain a unificate code also in case of only 1 cell segmentation
    try:
      n_layer_auto = mask.shape[2]
    except:  # case in which no cells or only 1 cell has been segmented
      if mask.sum()==0:  # no found cell case
        n_layer_auto = 0
      else:  # one cell case
        n_layer_auto = 1
        mask = np.expand_dims(mask,2)  # dimensions expansion is needed to mantain a unificate code also in case of only 1 cell segmentation
    ## current patch performance calculation
    perf = np.zeros((max(n_layer_manu,n_layer_auto),3))  # array that will contain performances for the current patch mask-matching predictions
    if n_layer_manu and n_layer_auto:  # if at least one cell has been segmented in both manual mask and automatic mask
      IoU_WC = np.zeros((n_layer_auto,n_layer_manu))  # array that will contain IoU of whole cell all possible combinations of 'single layer manual mask' | 'single layer predicted mask'
      ### whole cell level IoU of all possible combinations
      for o1 in range(n_layer_manu):
        for o2 in range(n_layer_auto):
          IoU_WC[o2,o1] = ((manu[:,:,o1]).astype(bool)*(mask[:,:,o2]).astype(bool)).sum()/((manu[:,:,o1]).astype(bool)+(mask[:,:,o2]).astype(bool)).sum()  #IoU of current combination 
      ### case in which automatic prediction found at least equal number of 
      ### cells of those identified in the manual annotations
      if n_layer_auto >= n_layer_manu: 
        order = np.argmax(IoU_WC,axis=0)  # finding most matching automatically segmented cell mask for each manually segmented cell
        #### solving manual mask - automatic mask matching redundancies: it may happen that one single-cell segmetation of one of the two types is matched with more than one single-cell segmentation of the other type, but obviously matchings have to be non redundant
        _,inv_ind,counts = np.unique(order,return_inverse=True, return_counts=True)
        if (counts>1).any():
          for a in range(max(inv_ind)+1):
            if (inv_ind==a).sum() > 1:
              w2d = (inv_ind==a)*~(IoU_WC.max(axis=0)==max(IoU_WC.max(axis=0)[inv_ind==a]))
              order = np.delete(order,w2d)
              inv_ind = np.delete(inv_ind,w2d)
              IoU_WC = np.delete(IoU_WC,w2d,1)
              manu = manu[:,:,~w2d]
        IoU_WC = np.max(IoU_WC,axis=0);  # IoU whole cell for all matched cell pairs
        IoU_N = np.sum((manu==255)*(mask[:,:,order]==255),axis=(0,1))/np.sum((manu==255)+(mask[:,:,order]==255),axis=(0,1))  # IoU nucleus for all matched cell pairs
        IoU_C = np.sum((manu==128)*(mask[:,:,order]==128),axis=(0,1))/np.sum((manu==128)+(mask[:,:,order]==128),axis=(0,1))  # IoU cytoplasm for all matched cell pairs
      ### case in which automatic prediction found a smaller number of cells of 
      ### those identified in the manual annotations
      else:
        order = np.argmax(IoU_WC,axis=1)
        #### solving manual mask - automatic mask matching redundancies: it may happen that one single-cell segmetation of one of the two types is matched with more than one single-cell segmentation of the other type, but obviously matchings have to be non redundant
        _,inv_ind,counts = np.unique(order,return_inverse=True, return_counts=True)
        if (counts>1).any():
          w2m_WC = np.ones(len(order),dtype=bool)
          for a in range(max(inv_ind)+1):
            if (inv_ind==a).sum() > 1:
              w2d = (inv_ind==a)*~(IoU_WC.max(axis=1)==max(IoU_WC.max(axis=1)[inv_ind==a]))
              order = np.delete(order,w2d)
              inv_ind = np.delete(inv_ind,w2d)
              IoU_WC = np.delete(IoU_WC,w2d,0)
              mask = mask[:,:,~w2d]
        IoU_WC = np.max(IoU_WC,axis=1)  
        IoU_N = np.sum((manu[:,:,order]==255)*(mask==255),axis=(0,1))/np.sum((manu[:,:,order]==255)+(mask==255),axis=(0,1))  
        IoU_C = np.sum((manu[:,:,order]==128)*(mask==128),axis=(0,1))/np.sum((manu[:,:,order]==128)+(mask==128),axis=(0,1))  
      perf = np.stack((IoU_N.T,IoU_C.T,IoU_WC.T),axis=1)  # performances on matched cells 
    erroneous_cells_vl.append(n_layer_auto-(perf[:,0]>0).sum()) 
    missed_cells_vl.append(n_layer_manu-(perf[:,0]>0).sum())
    dinofcep_vl.append(abs(n_layer_manu-n_layer_auto))   
    perf_vl = np.concatenate((perf_vl,perf,np.zeros((erroneous_cells_vl[-1]+missed_cells_vl[-1],3))),axis=0)  # adding current patch performances to global ones 

# performances printing
cell_number_diff_vl = np.array((erroneous_cells_vl,missed_cells_vl,dinofcep_vl),dtype=np.int8).T
print(f"\n\n\nPERFORMNACES ON VALIDATION SET as IoU nuclei | IoU cytoplasm | IoU whole cell:\n\n{perf_vl}\n\nmean and std: {np.mean(perf_vl,axis=0)} and {np.std(perf_vl,axis=0)}")
print(f"\n\n\n\nINVENTED CELLS | MISSED CELLS | DIFFERENCE IN # OF SEGMNETED CELLS:\n\n{cell_number_diff_vl}\n\n mean: {np.mean(cell_number_diff_vl,axis=0)}")
print(f"\n\n\n\nPERFORMNACES only on currectly detected MM cells as IoU nuclei | IoU cytoplasm | IoU whole cell:\n\nmean and std: {np.mean(perf_vl[perf_vl[:,0]>0,:],axis=0)} and {np.std(perf_vl[perf_vl[:,0]>0,:],axis=0)}")

100%|██████████| 50/50 [07:33<00:00,  9.07s/it]




PERFORMNACES ON VALIDATION SET as IoU nuclei | IoU cytoplasm | IoU whole cell:

[[9.35563786e-01 8.91678821e-01 9.66853901e-01]
 [8.89874023e-01 8.85069368e-01 9.68911453e-01]
 [9.20965452e-01 9.13865437e-01 9.73672688e-01]
 [8.22447738e-01 0.00000000e+00 4.25055344e-01]
 [9.23093521e-01 8.58450522e-01 9.55221033e-01]
 [9.38542089e-01 9.25852196e-01 9.79643324e-01]
 [9.56319643e-01 9.07974963e-01 9.49777009e-01]
 [9.73245288e-01 8.50059575e-01 9.18979934e-01]
 [9.59025188e-01 8.75486813e-01 9.22295766e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [9.10918491e-01 9.29313040e-01 9.50522420e-01]
 [9.68492270e-01 9.18943435e-01 9.62272520e-01]
 [9.73609438e-01 8.63739249e-01 9.25378788e-01]
 [8.79670178e-01 8.90923017e-01 9.60917844e-01]
 [9.16259732e-01 8.80321300e-01 9.27279267e-01]
 [9.26716877e-01 8.18632217e-01 9.10932229e-01]
 [9.32904387e-01 8.88760945e-01 9.41152470e-01]
 [8.77341635e-01 8.65471044e-01 9.32025992e-01]
 [9.28979592e-01 8.28187578e-01 9.17018293e-01]
 [0.0

* Test set

In [ ]:
# PREDICTIONS ON TEST SET

# Path
ts_IMGS_path = os.path.join('test','images')                              

# list of all images on which to apply the U-NET
ts_images = os.listdir(ts_IMGS_path)
for id_ in tqdm(ts_images, total=len(ts_images)):
    
    # load and resizing of stained image
    img = imread(ts_IMGS_path+'/'+id_)
    ori_shape = img.shape
    img = img_as_ubyte(resize(img,(rsz,rsz)))

    # color normalization
    img = htk.preprocessing.color_normalization.reinhard(img, mean_ref, std_ref)

    # apply ensembled models for prediction (ensembling is done at probability map level)
    hm = np.zeros((img.shape[0],img.shape[1],2),dtype=np.float32)
    for i in range(len(models)):
        hm = hm + models[i].predict(img[np.newaxis,:])[0,:,:,1:]  # notice reshaping of img to feed for net format is done here in fly  
    
    # post-processing
    ## post-processimg: thresholding
    mask0 = np.zeros((hm.shape[0],hm.shape[1]),dtype=np.uint8)
    mask0[hm[:,:,0]>1.35] = 128
    mask0[hm[:,:,1]>1.35] = 255  # predictions on nuclei are more accurate in terms of edges, so it is ok to eventually overwrite cytoplasm segmentations w/ nucleus segmentation
    ## post-processing: morphological refinement operations
    mask0 = morphology.area_closing(mask0,area_threshold=200)  # small holes removal
    mask0 = morphology.area_opening(mask0,area_threshold=100)  # small objects removal
    ## post-processing: resizing to original shape
    mask0 = img_as_ubyte(resize(mask0,(ori_shape[0],ori_shape[1])))
    mask0[mask0<80] = 0
    mask0[(mask0>=80)*(mask0<=175)] = 128
    mask0[mask0>175] = 255
    ## post-processing: removal of too small nuclei 
    nuclei,_ = ndimage.label(mask0==255)
    for n in range(1,nuclei.max()+1):
      if (nuclei==n).sum() < 0.003*mask0.shape[0]*mask0.shape[1]: mask0[nuclei==n] = 0
    ## post-processing: eventual watershed (hypothesis of each cytoplasm being 
    ## connected to corresponding nuclues in at least 1 pixel is done) and 
    ## storing each segmented cell on a different layer
    mask = np.zeros((mask0.shape[0],mask0.shape[1],0),dtype=np.uint8)  # initialization of the final N-layer mask
    labeled, n_trees = ndimage.label((mask0==128)+(mask0==255))  # finding connected components of thresholded mask
    for i in range(1,n_trees+1):
      _,n_nuc = ndimage.label((labeled==i)*(mask0==255))  # finding number of nuclei in the currently passed connected component (CC)
      if n_nuc > 1:   # case in which there is than 1 nucleus in the current CC, then the CC may be a cluster of cells --> watersheding is applied
        dist_transform = ndimage.distance_transform_edt(labeled==i)  # distance transform of the CC
        dst_for_mark = ndimage.distance_transform_edt((labeled==i)*(mask0==255)) # distance trasform on nuclei of the CC
        lab_for_mark,_ = ndimage.label((labeled==i)*(mask0==255))  # labeling nuclei of the CC
        mark_coords = peak_local_max(dst_for_mark, labels=lab_for_mark, num_peaks_per_label=1)  # finding markers' coordinate for subsequent watershed
        markers = np.zeros(dist_transform.shape, dtype=bool); markers[tuple(mark_coords.T)] = True; markers, _ = ndimage.label(markers)  # markers
        separated = watershed(-dist_transform, markers, mask=labeled==i)  # actual watersheding
        ### nuclei restoring (watersheding could have -erroneously- cut in more 
        ### parts some nucleus)
        restore = False  # nuclei restoring flag
        for j in range(1,separated.max()+1):  # cycling on the identified cells of the cluster
          curr = np.zeros_like(mask0); curr[separated==j] = mask0[separated==j] # segmentation of current cell
          mask = np.append(mask,curr[:,:,np.newaxis],axis=2)
          #### checking if nuclei restoring is needed 
          _,n_nuc2 = ndimage.label(curr==255)
          if n_nuc2 > 1: restore = True   
        if restore:   # if restoring is needed
          rank = np.zeros((n_nuc,n_nuc))  # array that will contain ranking of appartenence for each nuclei (on columns) to each watersheded object
          #### nuclei-matching ranking
          for k in range(1,n_nuc+1):  # loop over nuclei
             for l in range(1,n_nuc+1):  # loop over watersheded objects
               rank[l-1,k-1] = ((lab_for_mark==k)*(mask[:,:,-l]==255)).sum()/(lab_for_mark==k).sum()  
          rank = np.argmax(rank,0)+1  # array containing for each nucleus the watersheded object containing it at greatest %
          #### actual restoring
          for k in range(n_nuc):
            mask[lab_for_mark==k+1,-rank[k]] = 255
            mask[lab_for_mark==k+1,-1:-rank[k]:-1] = 0; mask[lab_for_mark==k+1,-rank[k]-1:-n_nuc-1:-1] = 0;
      else:  # case in which watersheding is not needed
        curr = np.zeros_like(mask0); curr[labeled==i] = mask0[labeled==i] # segmentation of current cell
        mask = np.append(mask,curr[:,:,np.newaxis],axis=2)
    
    # removing layers in which no nucleus has been found (minimum requirement to
    # have a segmentation is presence of nucleus)
    w2m = np.sum(mask==255,axis=(0,1)).astype(bool); mask = mask[:,:,w2m]  
    
    # if no cells have been segmneted, an empty single layer mask is saved [2-D 
    # empty mask], else if only one cell has been found, then the mask is 
    # "compressed" to 2-D  
    if mask.size == 0:  # no cell case
      mask = np.zeros((mask0.shape[0],mask0.shape[1]),dtype=np.uint8)
    elif mask.shape[2] == 1:  # 1 cell case
      mask = np.squeeze(mask)

    # saving mask: please, to read automatic annotations use 'imread' method of 
    # skimage library and not the one of imagecodecs library 
    imsave(os.path.join(save_ts,id_),mask,check_contrast=False)  

In [ ]:
# SAVINGS

# compression
!apt-get install rar  # installing necessary library
!rar a "train" "train_auto"
!rar a "validation" "validation_auto"
!rar a "test" "test_auto"

# saving in 03_PREDICTED MASKS
import shutil
shutil.copyfile("train.rar", "/content/drive/MyDrive/EIM challenge_gruppo_FA_DO_PA_PA/03_PREDICTED MASKS/train.rar")
shutil.copyfile("validation.rar", "/content/drive/MyDrive/EIM challenge_gruppo_FA_DO_PA_PA/03_PREDICTED MASKS/validation.rar")
shutil.copyfile("test.rar", "/content/drive/MyDrive/EIM challenge_gruppo_FA_DO_PA_PA/03_PREDICTED MASKS/test.rar")